# TensorFlow for Poets Codelab

This codelab was produced by Google. The original can be found here:

https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/?utm_campaign=chrome_series_machinelearning_063016&utm_source=gdev&utm_medium=yt-desc#0

## Start TensorBoard

Before starting the training, launch tensorboard in the background. TensorBoard is a monitoring and inspection tool included with tensorflow. You will use it to monitor the training progress.

`tensorboard --logdir tf_files/training_summaries &`

## Configuration Settings

Pick the following configuration options:

Input image resolution: 128,160,192, or 224px. Unsurprisingly, feeding in a higher resolution image takes more processing time, but results in better classification accuracy. We recommend 224 as an initial setting.

The relative size of the model as a fraction of the largest MobileNet: 1.0, 0.75, 0.50, or 0.25. We recommend 0.5 as an initial setting. The smaller models run significantly faster, at a cost of accuracy.

With the recommended settings, it typically takes only a couple of minutes to retrain on a laptop. You will pass the settings inside Linux shell variables. Set those shell variables as follows:

`IMAGE_SIZE=224
ARCHITECTURE="mobilenet_0.50_${IMAGE_SIZE}"`

## Investigate the retraining script

The retrain script is part of the tensorflow repo, but it is not installed as part of the pip package. So for simplicity I've included it in the codelab repository. You can run the script using the python command. Take a minute to skim its "help".

`python -m scripts.retrain -h`

## Run the training

As noted in the introduction, Imagenet models are networks with millions of parameters that can differentiate a large number of classes. We're only training the final layer of that network, so training will end in a reasonable amount of time.

Start your retraining with one big command (note the --summaries_dir option, sending training progress reports to the directory that tensorboard is monitoring):

`python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/"${ARCHITECTURE}" \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture="${ARCHITECTURE}" \
  --image_dir=tf_files/flower_photos`

This script downloads the pre-trained model, adds a new final layer, and trains that layer on the flower photos you've downloaded. 

ImageNet does not include any of these flower species we're training on here. However, the kinds of information that make it possible for ImageNet to differentiate among 1,000 classes are also useful for distinguishing other objects. By using this pre-trained network, we are using that information as input to the final classification layer that distinguishes our flower classes.

## Using the Retrained Model

The retraining script writes data to the following two files:

- tf_files/retrained_graph.pb, which contains a version of the selected network with a final layer retrained on your categories.

- tf_files/retrained_labels.txt, which is a text file containing labels.


### Classifying an image

The codelab repo also contains a copy of tensorflow's label_image.py example, which you can use to test your network. Take a minute to read the help for this script:

`python -m  scripts.label_image -h`

As you can see, this Python program takes quite a few arguments. The defaults are all set for this project, but if you used a MobileNet architecture with a different image size you will need to set the --input_size argument using the variable you created earlier: --input_size=${IMAGE_SIZE}.

`python -m scripts.label_image \
    --graph=tf_files/retrained_graph.pb  \
    --image=tf_files/flower_photos/daisy/21652746_cc379e0eea_m.jpg`

Now, let's run the script on this image of a daisy:

`python -m scripts.label_image \
    --graph=tf_files/retrained_graph.pb  \
    --image=tf_files/flower_photos/daisy/21652746_cc379e0eea_m.jpg`

Each execution will print a list of flower labels, in most cases with the correct flower on top (though each retrained model may be slightly different).

You might get results like this for a daisy photo:

`daisy (score = 0.99071)
sunflowers (score = 0.00595)
dandelion (score = 0.00252)
roses (score = 0.00049)
tulips (score = 0.00032)`

This indicates a high confidence (~99%) that the image is a daisy, and low confidence for any other label.

You can use label_image.py to classify any image file you choose, either from your downloaded collection, or new ones. You just have to change the `--image` file name argument to the script.

## Trying Other Hyperparameters

The retraining script has several other command line options you can use.

You can read about these options in the help for the retrain script:

`python -m scripts.retrain -h`

Try adjusting some of these options to see if you can increase the final validation accuracy.

For example, the `--learning_rate` parameter controls the magnitude of the updates to the final layer during training. So far we have left it out, so the program has used the default learning_rate value of 0.01. If you specify a small learning_rate, like 0.005, the training will take longer, but the overall precision might increase. Higher values of learning_rate, like 1.0, could train faster, but typically reduces precision, or even makes training unstable.

You need to experiment carefully to see what works for your case.

It is very helpful for this type of work if you give each experiment a unique name, so they show up as separate entries in TensorBoard.

It's the `--summaries_dir` option that controls the name in tensorboard. Earlier we used:

`--summaries_dir=training_summaries/basic`

TensorBoard is monitoring the contents of the training_summaries directory, so setting `--summarys_dir` to training_summaries or any subdirectory of training_summaries will work.

You may want to set the following two options together, so your results are clearly labeled:

`--learning_rate=0.5
--summaries_dir=training_summaries/LR_0.5`